In [35]:
import os,sys
import numpy as np
import sympy as sym
import sys
sym.init_printing()
sys.path.append(os.path.join(os.path.pardir, 'python'))
from tcg_slb.phasediagram.scipy import ScipyPDReactiveODE
from tcg_slb.phasediagram.base import PDReactiveGrid, PDReactiveGridDiagnostics, BasePDReactiveODE
from tcg_slb.base import *
import pickle
from pathlib import Path

reference= 'ec_downwelling'


In [36]:
def get_reaction(lib=''):
    pv = repr(sys.version_info.major)+'.'+repr(sys.version_info.minor)
    sys.path.append(os.path.join(os.path.pardir, 'database', 'install', lib, 'lib', 'python'+pv, 'site-packages/')) # the final slash is necessary!
    tcgdb = __import__('py_'+lib)
    func = getattr(tcgdb,lib) 
    rxn = func() # <-- this should work!
    return rxn

def get_pickle_path(rxnObj): 
    return Path('output', rxnObj.name() + '.pickle')

def load_grid(reactionObject):
    filename = get_pickle_path(reactionObject)
    with open(filename, 'rb') as pfile:
        bdfgrid = pickle.load(pfile)
    return bdfgrid

def save_grid(bdfgrid, reactionObject):
    filename = get_pickle_path(reactionObject)
    filename.parent.mkdir(exist_ok=True, parents=True)
    with open(filename, 'wb+') as pfile:
        pickle.dump(bdfgrid, pfile)

def solve_reaction_grid(reactionObject, mi0=None, Cik0=None, T_range=None, p_range=None, filename=None, reload=False, save=False, odeClass=ScipyPDReactiveODE,Da=None):
    if reload:
        return load_grid(reactionObject)

    i0 = 2 # doesn't matter as long as you pass cik0
    end = 1e5 # end time

    bdfgrid = PDReactiveGrid()

    bdfgrid.solve(reactionObject, ScipyPDReactiveODE, i0, ['T', 'p'], T_range, p_range, end, Cik0=Cik0, mi0=mi0)

    if save:
        save_grid(bdfgrid, reactionObject)

    return bdfgrid

def plot_reaction_grid(rxn, bdfgrid, plot_phases=True,figure_background=None, figure_xlim=[273.15,1273.15], figure_ylim=[0.0,5.0]):
    import matplotlib.pyplot as plt

    def decorate(pdrgd):
        def new_setup_axes(self,axi):
            if(figure_background is not None):
                img = plt.imread(figure_background)
                ip = axi.imshow(img)
            axi.axis('off')
            ax = axi.inset_axes([0.001,0.006,0.995,0.991])
            ax.patch.set_alpha(0.0)
            
            ax.set_xlabel("Temperature (K)")
            ax.set_ylabel("Pressure (GPa)")
            ax.set_xlim(figure_xlim)
            ax.set_ylim(figure_ylim)
            return ax
    
        #replace the display with newdisplay
        pdrgd.setup_axes = new_setup_axes
    
        #return the modified student 
        return pdrgd

    bdfdiag = decorate(PDReactiveGridDiagnostics)(rxn,bdfgrid) 
    s=bdfdiag.plot_rho()
    s.set_clim([25., 35.])
    s.set_cmap('jet')

    if plot_phases:
        bdfdiag.plot_phases()
    return bdfdiag

def run_reaction(rxnName=None, mi0=None, Cik0=None, t_range=None, p_range=None, reload=False, save=False,t0=900,p0=2.0,**kwargs):

    rxn = get_reaction(rxnName)

    # initial temperature, pressure and phase volume fraction
    Ti = t0 # kelvin
    pi = GPa2Bar(p0) # bars

    try:
        ode = ScipyPDReactiveODE(rxn)
        end_t = 1000.0
        ode.solve(Ti,pi,mi0,Cik0,end_t,**kwargs)
        display(ode.stime)
        display(ode.final_phases(1.e-2))
        ode.plot()
    except:
        pass

    grid = solve_reaction_grid(rxn, mi0=mi0, Cik0=Cik0, T_range=t_range, p_range=p_range, reload=reload, save=save,**kwargs)
    plot_reaction_grid(rxn,grid, plot_phases=True)
    return rxn, grid

In [37]:
X_Mg_garnet = 9.888
X_Ca_garnet = 10.549
X_Fe_garnet = 16.369
X_tot_garnet = X_Mg_garnet+X_Ca_garnet+X_Fe_garnet

X_py = X_Mg_garnet/X_tot_garnet
X_alm = X_Fe_garnet/X_tot_garnet
X_gr = X_Ca_garnet/X_tot_garnet

print(X_py, X_alm, X_gr)

X_Na_cpx = 6.326

X_Mg_cpx = 9.451
N_Mg_cpx = 0.498

X_Ca_cpx = 14.948
N_Ca_cpx = 0.566

X_Fe_cpx = 2.304
N_Fe_cpx = 0.068

X_Al_cpx = 10.407

X_tot_cpx = X_Na_cpx + X_Mg_cpx + X_Ca_cpx + X_Fe_cpx + X_Al_cpx

N_Ca_di = N_Mg_cpx
X_Ca_di = (N_Ca_di / N_Ca_cpx) * X_Ca_cpx
X_Ca_he = X_Ca_cpx - X_Ca_di

X_di = (X_Mg_cpx + X_Ca_di) / X_tot_cpx
X_jd = (X_Na_cpx + X_Al_cpx) / X_tot_cpx
X_he = 1 - X_di - X_jd

print(X_di, X_jd, X_he)

0.26865185024180843 0.44473727109710376 0.28661087866108786
0.5203777329514817 0.38523344691039685 0.09438882013812144


In [38]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, LinearConstraint, Bounds
from scipy.integrate import solve_ivp

# convert mole fractions between moles of Si2O4 and SiO2
si2_to_si = lambda xq : 2.*xq/(1.+xq)
si_to_si2 = lambda xq : xq/(2.-xq)

# convert temperatures
to_Kelvin = lambda T : T + 273.15
to_Celsius = lambda T : T - 273.15

phases = [
    'Clinopyroxene',
    'Orthopyroxene',
    'Quartz',
    'Feldspar', 
    'Garnet', 
    'Kyanite',
]

ems = [
    'Diopside', 'Hedenbergite', 'Clinoenstatite', 'CaTschermaks', 'Jadeite',
    'Enstatite', 'Ferrosilite', 'MgTschermaks', 'OrthoDiopside',
    'Quartz',
    'Anorthite','Albite',
    'Pyrope', 'Almandine', 'Grossular', 'MgMajorite', 'NaMajorite',
    'Kyanite'
]

N = len(phases) # num phases
K = len(ems) # num endmembers

def f(t,u,rxn,scale,epsilon=1.e-3,verbose=True):
    '''
    return rhs of the dimensionless 1-D isentropic upwelling equations

    Parameters
    ----------

    t: float
        time
    u: array
        solution array [ F, c_(Lq)^{Si2O4}, T, P ]
    rxn:  ThermoCodegen Reaction object
    scale: dict
        dictionary containing scales for T,P,rho
    epsilon: float
        regularization parameter for composition equation


    '''

    
    # Extract variables

    F = u[:N] # phase fracs
    C = u[N:N+K]
    T = u[N+K]
    P = u[N+K+1]

    # scale Temperature and pressure
    Ts = scale['T']*T
    Ps = scale['P']*P

    # calculate thermodynamic properties from the rxn object
    gamma_i = ode.Gammai(Ts,Ps,C,F)
    gamma_ik = ode.Gamma_ik(T,Ps,C,F)
    if verbose:
        print(gamma_i)
        print(gamma_ik)

    # phase properties
    rho = np.array(rxn.rho(Ts, Ps, C))
    alpha = np.array(rxn.alpha(Ts,Ps,C))
    cp = np.array(rxn.Cp(Ts, Ps , C))
    s = np.array(rxn.s(Ts, Ps, C))
    v = F/rho

    # mean properties
    rho_bar = 1./sum(v)
    alpha_bar = v.dot(alpha)*rho_bar
    cp_bar = F.dot(cp)
    s_bar = F.dot(s)

    A = scale['P']/scale['rho']

    dFdz = gamma_i # change in phases from reactions
    dTdz = - T/cp_bar * ( -A*alpha_bar  - s.dot(gamma_i)  ) # added a minus sign?
    dPdz = rho_bar/scale['rho'] # removed a minus sign because we are downwelling?

    if verbose:
        print(T/cp_bar, A*alpha_bar, s.dot(gamma_i))

    du = np.empty(u.shape)

    du[N+K:] = np.array([dTdz, dPdz])

    return du


In [43]:
rxn = get_reaction('ec_simple_slb_rx')

T0 = 300.15  # Kelvin
P0 = 500  # bars

# mass fractions of the end-members
# note: * = thermodynamic endmember, set to zero
C0 = [
    [0.25, 0.25, 0., 0., 0.5],  # di, hed, *cEn, *cats, jd
    [0.5, 0.5, 0., 0.],  # en, fs, *mats, *oDi
    [1.],  # quartz
    [0.25, 0.75],  # an, ab
    [0.4, 0.4, 0.2, 0., 0.],  # py, alm, gr, *mgmaj, *namaj
    [1.]  # ky
]

mi0 = [
    0.25, 0.25, 0., 0., 0.5,  # di, hed, *cEn, *cats, jd
    0.5, 0.5, 0., 0.,  # en, fs, *mats, *oDi
    1.,  # quartz
    0.25, 0.75,  # an, ab
    0.4, 0.4, 0.2, 0., 0.,  # py, alm, gr, *mgmaj, *namaj
    1.  # ky
]
C = C0.copy()

# Granulite facies
# mass fraction phases
Phi0 = [
    0.09,  # cpx
    0.24,  # opx
    0.08,  # quartz
    0.51,  # feldspar
    0.0,  # garnet
    0.07  # kyanite
]

X0 = rxn.C_to_X(C0)
print('C = ', C0)
print('X = ', X0)

rho = np.array(rxn.rho(T0, P0, C0))
F0 = rho*Phi0/rho.dot(Phi0)
print('F = ', F0)
rho = np.array(rxn.rho(T0, P0, C0))

Da = 100.
#rxn.set_parameter('Stot',Da)
rxn.list_parameters()

v = F0/rho
rho_bar = 1/sum(v)

# depth/distance scale in km
g = 9.81  # m/s^2
h = P0*1e5/(rho_bar*100.*g)/1000

scale = {'T': T0, 'P': P0, 'rho': rho_bar, 'h': h}
print(scale)

ode = BasePDReactiveODE(rxn)
ode.set_initial_params(T0,P0,F0,mi0)

def _rhs(t, u):
    du = ode.rhs_orig()
    T = u[-2]
    P = u[-1]
    # scale Temperature and pressure
    Ts = scale['T']*T
    Ps = scale['P']*P
    
    # phase properties
    rho = np.array(rxn.rho(Ts, Ps, C))
    alpha = np.array(rxn.alpha(Ts,Ps,C))
    cp = np.array(rxn.Cp(Ts, Ps , C))
    s = np.array(rxn.s(Ts, Ps, C))
    v = F/rho

    # mean properties
    rho_bar = 1./sum(v)
    alpha_bar = v.dot(alpha)*rho_bar
    cp_bar = F.dot(cp)
    s_bar = F.dot(s)
    du_new = np.zeros(len(du)+2)
    dTdz = - T/cp_bar * ( -A*alpha_bar  - s.dot(gamma_i)  ) # added a minus sign?
    dPdz = rho_bar/scale['rho'] # removed a minus sign because we are downwelling?
    du_new[-2] = dTdz
    du_new[-1] = dPdz

ode.rhs_orig = ode.rhs
ode.rhs = _rhs



C =  [[0.25, 0.25, 0.0, 0.0, 0.5], [0.5, 0.5, 0.0, 0.0], [1.0], [0.25, 0.75], [0.4, 0.4, 0.2, 0.0, 0.0], [1.0]]
X =  [[0.24903655334109495, 0.21737477070720712, 0.0, 0.0, 0.5335886759516979], [0.56788498937239, 0.43211501062761015, 0.0, 0.0], [1.0], [0.23906897642464334, 0.7609310235753567], [0.4429947132808382, 0.35877867493085736, 0.19822661178830436, 0.0, 0.0], [1.0]]
F =  [0.1026523  0.28668019 0.07118166 0.45340124 0.         0.08608461]
Parameters: 

R = 8.31446
T0 = 2000
{'T': 300.15, 'P': 500, 'rho': 30.103816884683624, 'h': 1.69308761701194}
u0 =  [1.02652299e-01 2.86680186e-01 7.11816597e-02 4.53401241e-01
 0.00000000e+00 8.60846142e-02 2.50000000e-01 2.50000000e-01
 0.00000000e+00 0.00000000e+00 5.00000000e-01 5.00000000e-01
 5.00000000e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00
 2.50000000e-01 7.50000000e-01 4.00000000e-01 4.00000000e-01
 2.00000000e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00
 3.00150000e+02 5.00000000e+02]


TypeError: Gamma_i(): incompatible function arguments. The following argument types are supported:
    1. (self: py_ec_simple_slb_rx.ec_simple_slb_rx, T: float, P: float, C: List[List[float]], Phi: List[float]) -> List[float]
    2. (self: py_ec_simple_slb_rx.ec_simple_slb_rx, T: float, P: float, C: List[List[float]], Phi: List[float], i: int) -> float

Invoked with: <py_ec_simple_slb_rx.ec_simple_slb_rx object at 0x177b69070>, 90090.02249999999, 250000.0, array([0.25, 0.25, 0.  , 0.  , 0.5 , 0.5 , 0.5 , 0.  , 0.  , 1.  , 0.25,
       0.75, 0.4 , 0.4 , 0.2 , 0.  , 0.  , 1.  ]), array([0.1026523 , 0.28668019, 0.07118166, 0.45340124, 0.        ,
       0.08608461])